In [2]:
# Import dependencies.
import pandas as pd
import numpy as np
from citipy import citipy
# Import the API key.
from config import weather_api_key
# Import the requests library.
import requests

## Imports that we used in the modules, but were not needed for this deliverable.
# import matplotlib.pyplot as plt
# Import the time module.
# import time
# Import the datetime module from the datetime library.
# from datetime import datetime
# Import linear regression from the SciPy stats module.
# from scipy.stats import linregress

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

787

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

#loop through cities in list
for i, city in enumerate(cities):
    
    #group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    #create URL endpoint for city
    city_url = url + '&q=' + city.replace(" ","+")
    
    #log the URL, record, and set numbers and the city
    print(f'Processing Record {record_count} of Set {set_count} | {city}')
    # add 1 to record count
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_dc = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        # city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_dc})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | lorengau
Processing Record 2 of Set 1 | amderma
City not found. Skipping...
Processing Record 3 of Set 1 | ayorou
Processing Record 4 of Set 1 | dudinka
Processing Record 5 of Set 1 | nueve de julio
Processing Record 6 of Set 1 | jamestown
Processing Record 7 of Set 1 | faya
Processing Record 8 of Set 1 | severo-kurilsk
Processing Record 9 of Set 1 | hit
Processing Record 10 of Set 1 | kapaa
Processing Record 11 of Set 1 | kalamazoo
Processing Record 12 of Set 1 | asosa
Processing Record 13 of Set 1 | pedasi
Processing Record 14 of Set 1 | tornio
Processing Record 15 of Set 1 | narsaq
Processing Record 16 of Set 1 | barrow
Processing Record 17 of Set 1 | ribeira brava
Processing Record 18 of Set 1 | salinopolis
Processing Record 19 of Set 1 | mataura
Processing Record 20 of Set 1 | bethel
Processing Record 21 of Set 1 | puerto ayora
Processing Record 22 of Set 1 | grand river south east
City not 

In [7]:
# Recall that we can convert a list of dictionaries to a Pandas DataFrame using df = pd.DataFrame(list with dictionaries).
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Lorengau,PG,-2.0226,147.2712,81.88,78,99,7.07,overcast clouds
1,Ayorou,NE,14.7308,0.9174,71.71,13,3,11.05,clear sky
2,Dudinka,RU,69.4058,86.1778,-3.39,94,100,13.87,overcast clouds
3,Nueve De Julio,AR,-35.4444,-60.8831,82.71,60,38,5.82,scattered clouds
4,Jamestown,US,42.0970,-79.2353,12.09,54,0,3.00,clear sky
5,Faya,SA,18.3851,42.4509,55.51,75,0,6.11,clear sky
6,Severo-Kurilsk,RU,50.6789,156.1250,23.79,77,92,18.37,light snow
7,Hit,IQ,33.6416,42.8251,49.55,61,38,3.53,scattered clouds
8,Kapaa,US,22.0752,-159.3190,80.58,67,0,3.00,clear sky
9,Kalamazoo,US,42.2506,-85.5500,20.95,36,75,5.75,broken clouds


In [8]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")